# Deep Learning with PyTorch: A 60 Minute Blitz

https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html

## Network Definition

In [ ]:
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader, TensorDataset
from torchviz import make_dot

sns.set()

In [ ]:
class ExampleNet(nn.Module):
    def __init__(self):
        super().__init__()
        # One input channel, 6 output channels, and 3x3 kernels
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # In features, out features
        self.lin1 = nn.Linear(1 * 6 * 6 * 16, 120)
        self.lin2 = nn.Linear(120, 84)
        self.lin3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        # Reshape into 1D vector
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = self.lin3(x)
        return x

    def num_flat_features(self, x):
        # all dimensions except the batch dimension
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


model = ExampleNet()
print(model)

In [ ]:
x = torch.randn(1, 1, 32, 32)
y = model(x)
print(y)

In [ ]:
make_dot(y, params=dict(model.named_parameters()))

In [ ]:
model.zero_grad()
y.backward(torch.randn(1, 10))

## Loss Functions

In [ ]:
y = model(x)
# a dummy target, for example
target = torch.randn(10)
# make it the same shape as output
target = target.view(1, -1)
# MSELoss returns a callable
criterion = nn.MSELoss()
loss = criterion(y, target)
print(loss)

## Backpropogation and Weight Updates

In [ ]:
model.zero_grad()  # zeroes the gradient buffers of all parameters

print("conv1.bias.grad before backward")
print(model.conv1.bias.grad)

loss.backward()

print("conv1.bias.grad after backward")
print(model.conv1.bias.grad)

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.01)

# One iteration of the training loop.
optimizer.zero_grad()
y = model(x)
loss = criterion(y, target)
loss.backward()
# Does the update
optimizer.step()